In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
import hdbscan
from scipy import stats
from tqdm import tqdm
from sklearn.cluster import DBSCAN
#from trackml.dataset import load_event, load_dataset
#from trackml.score import score_event
from IPython.display import clear_output
from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from tqdm import tqdm
from multiprocessing import Pool
from sklearn.neighbors import KDTree

In [52]:
cnt = 0
for event_id, hits, cells in load_dataset('../storage/track_ml_data/test.zip', parts=['hits', 'cells']):
#     if cnt == 1:
#         break
    print(event_id)
    cnt += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124


In [3]:
event_id

8181

In [2]:
cnt = 0
for event_id, hits, cells, particles, truth in load_dataset('../storage/track_ml_data/train_5.zip'):
    if cnt == 1:
        break
    cnt += 1

In [55]:
hits.to_csv('hit1.csv', index=False)

In [56]:
truth.to_csv('truth1.csv', index=False)

In [54]:
event_id

8181

In [50]:
c = [1.5,1.5,0.73,0.17,0.027,0.027] #[phi_coef,phi_coef,zdivrt_coef,zdivr_coef,xdivr_coef,ydivr_coef]
min_samples_in_cluster = 1


def merge(cl1, cl2, min_cnt): # merge cluster 2 to cluster 1
    d = pd.DataFrame(data={'s1':cl1,'s2':cl2})
    d['N1'] = d.groupby('s1')['s1'].transform('count')
    d['N2'] = d.groupby('s2')['s2'].transform('count')
    maxs1 = d['s1'].max()
    cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<20) &  (d['N2'].values>min_cnt))
    #cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<20) )

    s1 = d['s1'].values 
    s1[cond] = d['s2'].values[cond]+maxs1 
    return s1

def extract_good_hits(truth, submission):
    tru = truth[['hit_id', 'particle_id', 'weight']].merge(submission, how='left', on='hit_id')
    tru['count_both'] = tru.groupby(['track_id', 'particle_id']).hit_id.transform('count')    
    tru['count_particle'] = tru.groupby(['particle_id']).hit_id.transform('count')
    tru['count_track'] = tru.groupby(['track_id']).hit_id.transform('count')
    return tru[(tru.count_both > 0.5*tru.count_particle) & (tru.count_both > 0.5*tru.count_track)]

def fast_score(good_hits_df):
    return good_hits_df.weight.sum()





class Clusterer(object):
    def __init__(self):                        
        self.abc = []
          
    def initialize(self,dfhits):
        self.cluster = range(len(dfhits))
        
        
 
    def find_labels(self, params):
        hits, dz, z_shift, unroll_type = params

        hits = preprocess_hits(hits, z_shift)
        
        
        
        if unroll_type == 0:
            hits['a1'] = hits['a0'].values + np.nan_to_num(np.arccos(dz*hits['rt'].values))
        if unroll_type == 1:
            hits['a1'] = hits['a0'].values + dz*hits['rt'].values
        if unroll_type == 2:
            hits['a1'] = hits['a0'].values + dz*hits['z'].values                
                        

        hits['sina1'] = np.sin(hits['a1'].values)
        hits['cosa1'] = np.cos(hits['a1'].values)

        ss = StandardScaler()
        hits = ss.fit_transform(hits[['sina1','cosa1','zdivrt','zdivr','xdivr','ydivr']].values) 
        #dfs = scale_ignore_nan(dfh[['sina1','cosa1','zdivrt','zdivr','xdivr','ydivr']])
        hits = np.multiply(hits, c)
        new_cluster=DBSCAN(eps=0.0048,min_samples=1,metric='euclidean',n_jobs=1).fit(hits).labels_
        return new_cluster
        #merged_cluster = merge(merged_cluster, new_cluster)
        #result += [new_cluster]
#         if verbose == True:
#             sub = create_one_event_submission(0, hits, merged_cluster)
#             good_hits = extract_good_hits(truth, sub)
#             score_1 = fast_score(good_hits)
#             print('2r0_inverse:', ii*mm ,'. Score:', score_1)
                    
    
        
        
    def Hough_clustering(self,hits, epsilon, verbose=True): # [phi_coef,phi_coef,zdivrt_coef,zdivr_coef,xdivr_coef,ydivr_coef]
        #merged_cluster = self.cluster
        stepii = 0.000005
        count_ii = 0
        adaptive_eps_coefficient = 1
        
        result = []
        
        params = []
        for i in range(0, 3000):
            unroll_type = np.random.randint(0,3)
            if unroll_type == 0:
                dz = np.random.normal(0.0, 0.00035)
            elif unroll_type == 1:
                dz = np.random.normal(0.0, 0.00065)
            elif unroll_type == 2:
                dz = np.random.normal(0.0, 0.00085)

            #dz = 0
            z_shift = np.random.normal(0.0, 4.5)
            #z_shift = 0
            eps_new = epsilon + i*1*10**(-5)

            params.append((hits, dz, z_shift, unroll_type))
            
            
        
             
        pool = Pool(processes=8)
        result = []
        for i in tqdm(pool.imap(self.find_labels, params)):
            result += [i]
        pool.close()
                
            
        #self.cluster = merged_cluster
        return np.array(result)

def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

def preprocess_hits(h,dz):
    h['z'] =  h['z'].values + dz
    h['r'] = np.sqrt(h['x'].values**2+h['y'].values**2+h['z'].values**2)
    h['rt'] = np.sqrt(h['x'].values**2+h['y'].values**2)
    h['a0'] = np.arctan2(h['y'].values,h['x'].values)
    h['zdivrt'] = h['z'].values/h['rt'].values
    h['zdivr'] = h['z'].values/h['r'].values
    h['xdivr'] = h['x'].values / h['r'].values
    h['ydivr'] = h['y'].values / h['r'].values
    return h



In [51]:
c = [1.5,1.5,0.73,0.17,0.027,0.027] #[phi_coef,phi_coef,zdivrt_coef,zdivr_coef,xdivr_coef,ydivr_coef]
min_samples_in_cluster = 1

model = Clusterer()
#model.initialize(hits)

#hits_with_dz = preprocess_hits(hits, 0.055*i)

result = model.Hough_clustering(hits,epsilon=0.0048,verbose=True)





0it [00:00, ?it/s]



































1it [00:01,  1.73s/it]



9it [00:03,  2.82it/s]



15it [00:03,  4.54it/s]



17it [00:04,  3.65it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in arccos




22it [00:04,  4.61it/s]



25it [00:06,  4.11it/s]



27it [00:06,  4.35it/s]



33it [00:07,  4.44it/s]



35it [00:07,  4.56it/s]



41it [00:08,  4.65it/s]



42it [00:09,  4.61it/s]



49it [00:10,  4.79it/s]



50it [00:10,  4.74it/s]



57it [00:11,  4.92it/s]



58it [00:11,  4.86it/s]



65it [00:12,  5.03it/s]



66it [00:13,  4.93it/s]



73it [00:14,  5.12it/s]



74it [00:14,  5.00it/s]



81it [00:15,  5.20it/s]



82it [00:16,  5.06it/s]



89it [00:16,  5.26it/s]



90it [00:17,  5.10it/s]



94it [00:17,  5.29it/s]



97it [00:18,  5.32it/s]



99it [00:19,  5.21it/s]



103it [00:19,  5.37it/s]



105it [00:19,  5.32it/s]



107it [00:20,  5.19it/s]



113it [00:21,  5.36it/s]



114it [00:21,

741it [02:02,  6.05it/s]



744it [02:02,  6.06it/s]



746it [02:02,  6.07it/s]



748it [02:03,  6.06it/s]



749it [02:03,  6.05it/s]



752it [02:04,  6.06it/s]



755it [02:04,  6.05it/s]



757it [02:05,  6.04it/s]



760it [02:05,  6.06it/s]



763it [02:06,  6.05it/s]



765it [02:06,  6.04it/s]



768it [02:06,  6.06it/s]



770it [02:06,  6.07it/s]



772it [02:07,  6.06it/s]



774it [02:07,  6.05it/s]



777it [02:08,  6.07it/s]



779it [02:08,  6.05it/s]



781it [02:09,  6.04it/s]



785it [02:09,  6.07it/s]



787it [02:09,  6.05it/s]



789it [02:10,  6.04it/s]



793it [02:10,  6.07it/s]



795it [02:11,  6.05it/s]



797it [02:11,  6.04it/s]



802it [02:12,  6.07it/s]



804it [02:12,  6.06it/s]



806it [02:13,  6.04it/s]



811it [02:13,  6.05it/s]



813it [02:14,  6.04it/s]



818it [02:14,  6.07it/s]



820it [02:15,  6.06it/s]



822it [02:16,  6.04it/s]



827it [02:16,  6.06it/s]



829it [02:17,  6.03it/s]



835it [02:17,  6.06it/s]



837it [02:18,  6.03i

1671it [04:34,  6.08it/s]



1673it [04:35,  6.07it/s]



1678it [04:35,  6.08it/s]



1680it [04:36,  6.08it/s]



1681it [04:37,  6.07it/s]



1686it [04:37,  6.08it/s]



1688it [04:37,  6.08it/s]



1690it [04:38,  6.07it/s]



1694it [04:38,  6.08it/s]



1696it [04:38,  6.08it/s]



1698it [04:39,  6.07it/s]



1702it [04:39,  6.08it/s]



1704it [04:40,  6.08it/s]



1706it [04:41,  6.07it/s]



1710it [04:41,  6.08it/s]



1712it [04:41,  6.08it/s]



1714it [04:42,  6.06it/s]



1717it [04:42,  6.07it/s]



1719it [04:43,  6.06it/s]



1721it [04:43,  6.06it/s]



1722it [04:44,  6.06it/s]



1725it [04:44,  6.07it/s]



1728it [04:44,  6.07it/s]



1729it [04:45,  6.06it/s]



1730it [04:45,  6.06it/s]



1734it [04:45,  6.07it/s]



1736it [04:46,  6.07it/s]



1738it [04:46,  6.06it/s]



1741it [04:46,  6.07it/s]



1744it [04:47,  6.07it/s]



1746it [04:48,  6.06it/s]



1749it [04:48,  6.07it/s]



1752it [04:48,  6.07it/s]



1753it [04:48,  6.07it/s]



1754it [04:49,

2557it [07:01,  6.06it/s]



2563it [07:02,  6.07it/s]



2564it [07:02,  6.06it/s]



2568it [07:03,  6.07it/s]



2571it [07:03,  6.07it/s]



2573it [07:04,  6.06it/s]



2579it [07:05,  6.07it/s]



2580it [07:05,  6.06it/s]



2586it [07:05,  6.07it/s]



2589it [07:07,  6.06it/s]



2595it [07:07,  6.07it/s]



2597it [07:08,  6.06it/s]



2603it [07:09,  6.07it/s]



2605it [07:10,  6.06it/s]



2611it [07:10,  6.07it/s]



2613it [07:11,  6.05it/s]



2619it [07:11,  6.07it/s]



2621it [07:12,  6.05it/s]



2627it [07:13,  6.06it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in arccos




2629it [07:14,  6.05it/s]



2635it [07:14,  6.06it/s]



2638it [07:15,  6.05it/s]



2644it [07:16,  6.05it/s]



2646it [07:17,  6.05it/s]



2652it [07:18,  6.05it/s]



2653it [07:18,  6.05it/s]



2657it [07:18,  6.05it/s]



2660it [07:19,  6.05it/s]



2662it [07:20,  6.05it/s]



2668it [07:20,  6.05it/s]



2669it [07:21

In [52]:
result

array([[    0,     1,     2, ..., 93751, 93752, 93753],
       [    0,     1,     2, ..., 74027, 45017, 61490],
       [    0,     1,     2, ..., 92000, 92001, 92002],
       ...,
       [    0,     1,     2, ..., 77177, 77178, 77168],
       [    0,     1,     2, ..., 91188, 91189, 91190],
       [    0,     1,     2, ..., 81412, 81413, 81403]])

In [64]:
result.shape

(3000, 100287)

In [60]:
list(reversed(range(10)))

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [63]:
#result = result_53
labels = range(result.shape[1])

#for k in [18,16,14, 12, 10, 8, 6, 4, 0]:
#for k in list(reversed(range(21))):
for k in [0]:
    for i in range(len(result)):
        labels = merge(labels, result[i], k)
  
    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))
    #submission = extend(submission,hits)
    #submission = extend(submission,hits)

    #print(score_event(truth, submission))
    
    labels = submission['track_id'].values

0.5683607021832576


In [65]:
np.save('predicts/53_all', result)

In [ ]:
0.5683607021832576

In [46]:
def deeper(i):
    #print(i)
    np.random.seed(i)
    #result = p
    lock.acquire()
    np.random.shuffle(result)
    lock.release()
    labels = range(result.shape[1])

    for k in [0]:
        for i in range(len(result[:])):
            labels = merge(labels, result[i], k)

        submission = create_one_event_submission(0, hits['hit_id'].values, labels)
        print(score_event(truth, submission))
    return labels

In [39]:
    np.random.shuffle(result)


In [47]:
from threading import Lock
global lock
lock = Lock()

In [48]:
# p = []
# for i in range(10):
#     p.append((result))


pool = Pool(processes=8)
second = []
for i in tqdm(pool.imap(deeper, range(8))):
    second += [i]
pool.close()
second = np.array(second)





0it [00:00, ?it/s]

0.5498488299761319
0.552011738089534
0.5505511300983359






1it [00:45, 45.99s/it]

0.5499673718114294
0.5505025305903635
0.5495568990959527
0.5498816489367413






4it [00:46, 11.55s/it]

0.5490225923058363






8it [00:46,  5.78s/it]

In [28]:
result.shape

(3000, 100287)

In [49]:
result = np.array(second)
labels = range(result.shape[1])

for k in [0]:
    for i in range(len(result[:])):
        labels = merge(labels, result[i], k)

    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))

0.5635994418069967


In [58]:
second = []
for k in range(10):
    #result = np.vstack([res0, res1, res2])
    np.random.shuffle(result)

    #result = res0
    labels = range(result.shape[1])

    for k in [0]:
        for i in range(len(result[:])):
            labels = merge(labels, result[i], k)

        submission = create_one_event_submission(0, hits['hit_id'].values, labels)
        print(score_event(truth, submission))
        
    second += [labels]

0.5691456871396108
0.5677176649163719
0.5658043876731997
0.5684011651758079
0.5664038663741735
0.5675286869167122
0.5704528721377291
0.5662440397836588
0.5653050845520902
0.567874240902404


In [59]:
result = np.array(second)
labels = range(result.shape[1])

for k in [0]:
    for i in range(len(result[:])):
        labels = merge(labels, result[i], k)

    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))

0.5829938802054802


In [ ]:
0.5829938802054802
0.4869771922947843
0.4744072863229684

In [60]:
	def extend(submission,hits):

		df = submission.merge(hits,  on=['hit_id'], how='left')
		df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
		df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
		df = df.assign(arctan2 = np.arctan2(df.z, df.r))

		for angle in range(-180,180,1):

		    #print ('\r %f'%angle, end='',flush=True)
		    #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
		    df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

		    min_num_neighbours = len(df1)
		    if min_num_neighbours<4: continue

		    hit_ids = df1.hit_id.values
		    x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
		    r  = (x**2 + y**2)**0.5
		    r  = r/1000
		    a  = np.arctan2(y,x)
		    tree = KDTree(np.column_stack([a,r]), metric='euclidean')

		    track_ids = list(df1.track_id.unique())
		    num_track_ids = len(track_ids)
		    min_length=3

		    for i in range(num_track_ids):
		        p = track_ids[i]
		        if p==0: continue

		        idx = np.where(df1.track_id==p)[0]
		        if len(idx)<min_length: continue

		        if angle>0:
		            idx = idx[np.argsort( z[idx])]
		        else:
		            idx = idx[np.argsort(-z[idx])]


		        ## start and end points  ##
		        idx0,idx1 = idx[0],idx[-1]
		        a0 = a[idx0]
		        a1 = a[idx1]
		        r0 = r[idx0]
		        r1 = r[idx1]

		        da0 = a[idx[1]] - a[idx[0]]  #direction
		        dr0 = r[idx[1]] - r[idx[0]]
		        direction0 = np.arctan2(dr0,da0) 

		        da1 = a[idx[-1]] - a[idx[-2]]
		        dr1 = r[idx[-1]] - r[idx[-2]]
		        direction1 = np.arctan2(dr1,da1) 

	 
		        ## extend start point
		        ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
		        ns = np.concatenate(ns)
		        direction = np.arctan2(r0-r[ns],a0-a[ns])
		        ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]
	   
		        for n in ns:
		            df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

		        ## extend end point
		        ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
		        ns = np.concatenate(ns)

		        direction = np.arctan2(r[ns]-r1,a[ns]-a1)
		        ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 
		        
		        for n in ns:
		            df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
		#print ('\r')
		df = df[['event_id', 'hit_id', 'track_id']]
		return df


In [63]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.5997579258480263


In [64]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.6054562963314472


In [65]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.607168036563943


In [66]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.6081666694070661


In [34]:
100287 * 50000 * 8 / 1024/1024/1024

37.359818816185

In [80]:
def foo(t):
    i = 0
    model = Clusterer()
    model.initialize(hits)
    
    hits_with_dz = preprocess_hits(hits, 0.055*i)

    result = model.Hough_clustering(hits_with_dz,coef=c,epsilon=0.0048,min_samples=min_samples_in_cluster,
                       n_loop=300,verbose=True, unroll_type=t)
    
    #np.save('predicts/53/{}'.format(i), result)
    return result

In [81]:
%%time
from multiprocessing import Pool

c = [1.5,1.5,0.73,0.17,0.027,0.027] #[phi_coef,phi_coef,zdivrt_coef,zdivr_coef,xdivr_coef,ydivr_coef]
min_samples_in_cluster = 1

    
pool = Pool(processes=3)

import threading
lock = threading.Lock()

res = []
for df in tqdm(pool.imap(foo,  range(3))):
    res += [df]
    None

pool.close()

0it [00:00, ?it/s]

2r0_inverse: -0.0 . Score: 0.042406216
2r0_inverse: -0.0 . Score: 0.042406216
2r0_inverse: -0.0 . Score: 0.042406216
2r0_inverse: 0.0 . Score: 0.042406216
2r0_inverse: 0.0 . Score: 0.042406216
2r0_inverse: 0.0 . Score: 0.042406216
2r0_inverse: -5e-06 . Score: 0.054486685
2r0_inverse: -5e-06 . Score: 0.043635346
2r0_inverse: -5e-06 . Score: 0.043635346
2r0_inverse: 5e-06 . Score: 0.06586099
2r0_inverse: 5e-06 . Score: 0.04469385
2r0_inverse: 5e-06 . Score: 0.04469385
2r0_inverse: -1e-05 . Score: 0.07551442
2r0_inverse: -1e-05 . Score: 0.045647472
2r0_inverse: -1e-05 . Score: 0.045647472
2r0_inverse: 1e-05 . Score: 0.08650279
2r0_inverse: 1e-05 . Score: 0.046612665
2r0_inverse: 1e-05 . Score: 0.046612665
2r0_inverse: -1.5000000000000002e-05 . Score: 0.09945688
2r0_inverse: -1.5000000000000002e-05 . Score: 0.047836985
2r0_inverse: -1.5000000000000002e-05 . Score: 0.047836985
2r0_inverse: 1.5000000000000002e-05 . Score: 0.109854326
2r0_inverse: 1.5000000000000002e-05 . Score: 0.04847325
2r

2r0_inverse: 0.000145 . Score: 0.15379623
2r0_inverse: 0.000145 . Score: 0.15410146
2r0_inverse: -0.00015000000000000001 . Score: 0.35325712
2r0_inverse: -0.00015000000000000001 . Score: 0.15593237
2r0_inverse: -0.00015000000000000001 . Score: 0.1555512
2r0_inverse: 0.00015000000000000001 . Score: 0.355431
2r0_inverse: 0.00015000000000000001 . Score: 0.15891428
2r0_inverse: 0.00015000000000000001 . Score: 0.15828286
2r0_inverse: -0.000155 . Score: 0.35741624
2r0_inverse: -0.000155 . Score: 0.16113457
2r0_inverse: -0.000155 . Score: 0.16068685
2r0_inverse: 0.000155 . Score: 0.3592171
2r0_inverse: 0.000155 . Score: 0.16383024
2r0_inverse: 0.000155 . Score: 0.1631654
2r0_inverse: -0.00016 . Score: 0.36090294
2r0_inverse: -0.00016 . Score: 0.1660802
2r0_inverse: -0.00016 . Score: 0.16575865
2r0_inverse: 0.00016 . Score: 0.3627551
2r0_inverse: 0.00016 . Score: 0.16858879
2r0_inverse: 0.00016 . Score: 0.16833457
2r0_inverse: -0.00016500000000000003 . Score: 0.36462224
2r0_inverse: -0.0001650

2r0_inverse: -0.00028500000000000004 . Score: 0.28086716
2r0_inverse: -0.00029 . Score: 0.42287597
2r0_inverse: 0.00028500000000000004 . Score: 0.28384012
2r0_inverse: 0.00028500000000000004 . Score: 0.28371733
2r0_inverse: 0.00029 . Score: 0.4233568
2r0_inverse: -0.00029 . Score: 0.28495884
2r0_inverse: -0.00029 . Score: 0.28561917
2r0_inverse: -0.000295 . Score: 0.42434898
2r0_inverse: 0.00029 . Score: 0.2871241
2r0_inverse: 0.00029 . Score: 0.28751466
2r0_inverse: 0.000295 . Score: 0.42494118
2r0_inverse: -0.000295 . Score: 0.28954098
2r0_inverse: -0.000295 . Score: 0.28993925
2r0_inverse: -0.00030000000000000003 . Score: 0.42528918
2r0_inverse: 0.000295 . Score: 0.29190618
2r0_inverse: 0.000295 . Score: 0.29224542
2r0_inverse: 0.00030000000000000003 . Score: 0.42639908
2r0_inverse: -0.00030000000000000003 . Score: 0.29411232
2r0_inverse: -0.00030000000000000003 . Score: 0.29388714
2r0_inverse: -0.00030500000000000004 . Score: 0.4269308
2r0_inverse: 0.00030000000000000003 . Score: 0

2r0_inverse: -0.00042500000000000003 . Score: 0.37769976
2r0_inverse: -0.00043000000000000004 . Score: 0.45487872
2r0_inverse: 0.00042500000000000003 . Score: 0.37970322
2r0_inverse: 0.00042500000000000003 . Score: 0.37928325
2r0_inverse: 0.00043000000000000004 . Score: 0.4550925
2r0_inverse: -0.00043000000000000004 . Score: 0.38164276
2r0_inverse: -0.00043000000000000004 . Score: 0.38078827
2r0_inverse: -0.00043500000000000006 . Score: 0.45598668
2r0_inverse: 0.00043000000000000004 . Score: 0.38269126
2r0_inverse: 0.00043000000000000004 . Score: 0.3819823
2r0_inverse: 0.00043500000000000006 . Score: 0.45596743
2r0_inverse: -0.00043500000000000006 . Score: 0.38431907
2r0_inverse: -0.00043500000000000006 . Score: 0.3832742
2r0_inverse: -0.00044 . Score: 0.4563824
2r0_inverse: 0.00043500000000000006 . Score: 0.3859712
2r0_inverse: 0.00043500000000000006 . Score: 0.38451442
2r0_inverse: 0.00044 . Score: 0.45681188
2r0_inverse: -0.00044 . Score: 0.38696933
2r0_inverse: -0.00044 . Score: 0.

...

2r0_inverse: 0.001495 . Score: 0.5455566
2r0_inverse: -0.001495 . Score: 0.5468827


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:115: RuntimeWarning: invalid value encountered in arccos


2r0_inverse: 0.001495 . Score: 0.5467987


3it [09:07, 182.36s/it]

CPU times: user 1.12 s, sys: 1.74 s, total: 2.86 s
Wall time: 9min 7s


In [63]:
foo(0)

2r0_inverse: -0.0 . Score: 0.042406216
2r0_inverse: 0.0 . Score: 0.042406216
2r0_inverse: -5e-06 . Score: 0.043635346
2r0_inverse: 5e-06 . Score: 0.04469385
2r0_inverse: -1e-05 . Score: 0.045647472
2r0_inverse: 1e-05 . Score: 0.046612665
2r0_inverse: -1.5000000000000002e-05 . Score: 0.047836985
2r0_inverse: 1.5000000000000002e-05 . Score: 0.04855337
2r0_inverse: -2e-05 . Score: 0.04980075
2r0_inverse: 2e-05 . Score: 0.05173693
2r0_inverse: -2.5e-05 . Score: 0.052394383
2r0_inverse: 2.5e-05 . Score: 0.053064913
2r0_inverse: -3.0000000000000004e-05 . Score: 0.053885788
2r0_inverse: 3.0000000000000004e-05 . Score: 0.05630056
2r0_inverse: -3.5000000000000004e-05 . Score: 0.05682042
2r0_inverse: 3.5000000000000004e-05 . Score: 0.05745018
2r0_inverse: -4e-05 . Score: 0.058533702
2r0_inverse: 4e-05 . Score: 0.059605278
2r0_inverse: -4.5e-05 . Score: 0.060867604
2r0_inverse: 4.5e-05 . Score: 0.062744826
2r0_inverse: -5e-05 . Score: 0.06345058
2r0_inverse: 5e-05 . Score: 0.06475884
2r0_inverse:

2r0_inverse: 0.00042500000000000003 . Score: 0.37970322
2r0_inverse: -0.00043000000000000004 . Score: 0.38164276
2r0_inverse: 0.00043000000000000004 . Score: 0.38269126
2r0_inverse: -0.00043500000000000006 . Score: 0.38431907
2r0_inverse: 0.00043500000000000006 . Score: 0.3859712
2r0_inverse: -0.00044 . Score: 0.38696933
2r0_inverse: 0.00044 . Score: 0.38789105
2r0_inverse: -0.00044500000000000003 . Score: 0.38936892
2r0_inverse: 0.00044500000000000003 . Score: 0.39081007
2r0_inverse: -0.00045000000000000004 . Score: 0.3915373
2r0_inverse: 0.00045000000000000004 . Score: 0.3927735
2r0_inverse: -0.00045500000000000006 . Score: 0.39390746
2r0_inverse: 0.00045500000000000006 . Score: 0.3952009
2r0_inverse: -0.00046 . Score: 0.3964424
2r0_inverse: 0.00046 . Score: 0.39721307
2r0_inverse: -0.000465 . Score: 0.39826426
2r0_inverse: 0.000465 . Score: 0.3996883
2r0_inverse: -0.00047000000000000004 . Score: 0.40048936
2r0_inverse: 0.00047000000000000004 . Score: 0.4016503
2r0_inverse: -0.000475

2r0_inverse: -0.0008500000000000001 . Score: 0.50125384
2r0_inverse: 0.0008500000000000001 . Score: 0.5015348
2r0_inverse: -0.0008550000000000001 . Score: 0.5021733
2r0_inverse: 0.0008550000000000001 . Score: 0.5025332
2r0_inverse: -0.0008600000000000001 . Score: 0.50299704
2r0_inverse: 0.0008600000000000001 . Score: 0.50343025
2r0_inverse: -0.0008650000000000001 . Score: 0.50370675
2r0_inverse: 0.0008650000000000001 . Score: 0.5040174
2r0_inverse: -0.0008700000000000001 . Score: 0.50439143
2r0_inverse: 0.0008700000000000001 . Score: 0.504791
2r0_inverse: -0.000875 . Score: 0.5049782
2r0_inverse: 0.000875 . Score: 0.5054898
2r0_inverse: -0.00088 . Score: 0.5058568
2r0_inverse: 0.00088 . Score: 0.50631857
2r0_inverse: -0.000885 . Score: 0.5066137
2r0_inverse: 0.000885 . Score: 0.5066004
2r0_inverse: -0.0008900000000000001 . Score: 0.50682694
2r0_inverse: 0.0008900000000000001 . Score: 0.5071365
2r0_inverse: -0.0008950000000000001 . Score: 0.50732327
2r0_inverse: 0.0008950000000000001 . 

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:113: RuntimeWarning: invalid value encountered in arccos


2r0_inverse: -0.0009800000000000002 . Score: 0.5165943


...

2r0_inverse: 0.0049900000000000005 . Score: 0.3466348


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:113: RuntimeWarning: invalid value encountered in arccos


2r0_inverse: -0.004995 . Score: 0.34661934


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:113: RuntimeWarning: invalid value encountered in arccos


2r0_inverse: 0.004995 . Score: 0.34635964


In [76]:
res[1]

[array([    0,     1,     2, ..., 71796, 71797, 71790]),
 array([    0,     1,     2, ..., 71796, 71797, 71790]),
 array([    0,     1,     2, ..., 71690, 71691, 71685]),
 array([    0,     1,     2, ..., 71798, 71799, 71793]),
 array([    0,     1,     2, ..., 71524, 71525, 71517]),
 array([    0,     1,     2, ..., 71713, 12346, 71707]),
 array([    0,     1,     2, ..., 71540, 71541, 71532]),
 array([    0,     1,     2, ..., 71592, 71593, 71586]),
 array([    0,     1,     2, ..., 71526, 71527, 71518]),
 array([    0,     1,     2, ..., 71598, 71599, 18281]),
 array([    0,     1,     2, ..., 71624, 71625, 71617]),
 array([    0,     1,     2, ..., 71596, 71597, 71590]),
 array([    0,     1,     2, ..., 71563, 12912, 22993]),
 array([    0,     1,     2, ..., 71465, 71466, 71458]),
 array([    0,     1,     2, ..., 71662, 71663, 23003]),
 array([    0,     1,     2, ..., 71706, 71707, 71699]),
 array([    0,     1,     2, ..., 60681, 71752, 58892]),
 array([    0,     1,     2, ..

In [77]:
np.save('predicts/53_0', res[0])

In [78]:
np.save('predicts/53_1', res[1])

In [79]:
np.save('predicts/53_2', res[2])

In [51]:
event_id

124

In [ ]:
def foo(i):
    
    model = Clusterer()
    model.initialize(hits)
    
    hits_with_dz = preprocess_hits(hits, 0.055*i)

    result = model.Hough_clustering(hits_with_dz,coef=c,epsilon=0.0048,min_samples=min_samples_in_cluster,
                       n_loop=1000,verbose=True, unroll_type=0)
    
    #np.save('predicts/53/{}'.format(i), result)
    np.save('predicts/53/{}_{}'.format(event_id, i), model.cluster)
    
    return result

In [74]:
10 * 200 / 60 / 8

4.166666666666667

In [28]:
%%time
from multiprocessing import Pool

c = [1.5,1.5,0.73,0.17,0.027,0.027] #[phi_coef,phi_coef,zdivrt_coef,zdivr_coef,xdivr_coef,ydivr_coef]
min_samples_in_cluster = 1

for event_id, hits, cells in load_dataset('../storage/track_ml_data/test.zip', parts=['hits', 'cells']):
    
    pool = Pool(processes=8)

    import threading
    lock = threading.Lock()


    for df in tqdm(pool.imap(foo,  range(-100,101))):
        None

    pool.close()


#submission = create_one_event_submission(0, hits, model.cluster)


0it [00:00, ?it/s]

2r0_inverse: -0.0 . Score: 0.03905902
2r0_inverse: -0.0 . Score: 0.03905902
2r0_inverse: -0.0 . Score: 0.03953301
2r0_inverse: -0.0 . Score: 0.03992958
2r0_inverse: -0.0 . Score: 0.03992958
2r0_inverse: -0.0 . Score: 0.039404385
2r0_inverse: -0.0 . Score: 0.03925472
2r0_inverse: -0.0 . Score: 0.03970274
2r0_inverse: 0.0 . Score: 0.03992958
2r0_inverse: 0.0 . Score: 0.03905902
2r0_inverse: 0.0 . Score: 0.03905902



1it [00:06,  6.45s/it]

2r0_inverse: 0.0 . Score: 0.03970274
2r0_inverse: 0.0 . Score: 0.03953301
2r0_inverse: 0.0 . Score: 0.03992958
2r0_inverse: 0.0 . Score: 0.039404385
2r0_inverse: 0.0 . Score: 0.03925472



3it [00:06,  2.20s/it]

...

2r0_inverse: 0.0 . Score: 0.014537788
2r0_inverse: 0.0 . Score: 0.014537788



32it [00:24,  1.28it/s]


KeyboardInterrupt: 

Process ForkPoolWorker-19:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-27-3c294331f738>", line 8, in foo
    n_loop=1,verbose=True)
  File "<ipython-input-10-e3b05f6186b3>", line 118, in Hough_clustering
    new_cluster=DBSCAN(eps=eps_new,min_samples=min_samples,metric='euclidean',n_jobs=4).fit(dfs).labels_
  File "/usr/local/lib/python3.5/dist-packages/sklearn/cluster/dbscan_.py", line 284, in fit
    **self.get_params())
Process ForkPoolWorker-20:
  File "/usr/local/lib/python3.5/dist-packages/sklearn/cluster/dbscan_.py", line 145, in dbscan
    return_distance=False)
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/proc

KeyboardInterrupt
  File "/usr/local/lib/python3.5/dist-packages/sklearn/neighbors/base.py", line 625, in radius_neighbors
    return_distance=return_distance)
KeyboardInterrupt


In [9]:
np.save('predicts/53', model.cluster)

In [13]:
np.save('predicts/53_all', result)

In [66]:
result.shape

(201, 2, 100287)

In [19]:
200 * 10 / 60 

33.333333333333336

In [11]:
result = []
for i in range(-100,101):
    result += [np.load('predicts/53/{}.npy'.format(i))]

result = np.array(result)

In [22]:
result[:, 0, :]

array([[    0,     1,     2, ..., 72011, 72012, 72003],
       [    0,     1,     2, ..., 72009, 72010, 72001],
       [    0,     1,     2, ..., 72014, 72015, 72006],
       ...,
       [    0,     1,     2, ..., 71766, 71767, 71760],
       [    0,     1,     2, ..., 71749, 71750, 71744],
       [    0,     1,     2, ..., 71781, 71782, 71775]])

In [44]:
10*200 / 60 / 32

1.0416666666666667

In [41]:
np.load('predicts/53/{}.npy'.format(0))

array([[    0,     1,     2, ..., 71796, 71797, 71790],
       [    0,     1,     2, ..., 71796, 71797, 71790],
       [    0,     1,     2, ..., 71689, 71690, 71684],
       [    0,     1,     2, ..., 71799, 71800, 71794],
       [    0,     1,     2, ..., 71524, 71525, 71517],
       [    0,     1,     2, ..., 71712, 12346, 71706]])